In [11]:
from sklearn import datasets
from sklearn import preprocessing

In [14]:
iris = datasets.load_iris().data

In [15]:
iris_normalized = preprocessing.normalize(iris, norm='l2')

In [28]:
print("El minimo es %f - El máximo es %f " % (iris_normalized.min(), iris_normalized.max()))

El minimo es 0.014727 - El máximo es 0.860939 
